# Water Chemical Potential

By Alberto Meseguer

Here I will explain how my program works. There are few differences between the code in this notebook and the python code I have submitted. The main difference is that my python code works taking arguments from the execution line. These arguments define the names of the input and output files, and also define the sigma value for the gaussian normalization of the distribution of water molecules.

The goal of this program is to generate a .cube file which contains an isosurface regarding the water chemical potential for a set of molecular simulations. This is achieved by using the function writeVoxels, which will take a three dimensional array which contains information about the distribution of water molecules along the three dimensional space. The whole program is oriented to the obtaining of this array and to the execution of the writeVoxels function.

We will start by importing all the necesary packages:

In [14]:
import os
import sys
import re
import argparse
import itertools
import time
import pickle
from htmd import *
from numpy import *
from htmd.molecule.util import *

Now, the first step is to create a simulation list from the dataset of our simulations. In the python code, the input file is specified by an argument in the command line.

In [15]:
sims = simlist(glob("CXCL12-confAnalysis/*/"), glob("CXCL12-confAnalysis/*/" + "structure.pdb"), glob("CXCL12-confAnalysis/*/"))

Creating simlist: 100% (1/1) [#####################################] eta --:-- -


Then, we will define some variables that will be used along the program. 

In [16]:
num_wat = 0
t = 298
kb = 0.001987191
mol_list = list()
MA = list()

Now we will iterate on all the elements in the simulation list. This iteration has two objectives: getting the furthest point from the origin of coordinates with possitive sign in the x axis in the first frame, and creating a list containing structures with their respective trajectories. 

In [17]:
for element in sims:
	mol = Molecule(element.molfile)
	mol.read(element.trajectory)
	mol.wrap()
	mol.align("protein", refmol= Molecule(element.molfile))
	MA.append(amax(mol.coords[:,0,0])) 
	mol_list.append(mol)

MA contains the maximum coordinate for the x axis in the first frame for all simulations. We will use this value for making an estimation of the dimensions of the box where the simulation has been carried. For that, we asume that the simulations are carried in a cubical space. This will be useful for moving the simulation to a possitive quadrant in the three dimensional space. Avoiding negative coordinates will ease the calculations of the program. Thus, we get maximum, which will have the highest value of the X coordinate from all the simulations. 

In [19]:
maximum = max(MA) * 1.3
cord = int(floor(maximum*2.2))
mat = zeros((cord, cord, cord))
print(MA)
print(max(MA))
print(maximum)
print(cord)

[36.486942]
36.4869
47.4330249786
109


Then, now we will iterate on all molecules contained in our dataset, which have been appended to the list mol_list. Each element of the list will be included as an argument in the get_mat function. This function will return a three dimensional array, where each coordinate corresponds with a coordinate in three dimensional space. This array will contain the number of atoms of oxygen belonging to a molecule of water that have been found on that particular region of the three dimensional space. Then, we first define the function get_mat.

As we see, in this function we first move the box to a positive quadrant. Then, we filter all atoms of oxygen belonging to a molecule of water and we count the number of molecules of water that we have in the simulation considering all frames. Then, we iterate over all the frames, and inside of it we iterate for all oxygen atoms. For each oxygen atom we get it's coordinates, we round them and we add one to the value in the multidimensional array corresponding with the specified coordinates of the atom. For a proper performance, the matrix has to be big enough so all atoms of oxygen in the box should be included. However, negative values are not included as coordinates, that's why we move the whole molecule to a possitive quadrant. Just in case we have some unexpected molecule of water in negative or non considered coordinates, we have a warning comand that will inform us in case that this happens. Finally, the function returns the three dimension array and the number of molecules of water of the whole simulation.

In [20]:
def get_mat(element, mat, num_wat, maximum):
	mol = element
	mol.moveBy([maximum, maximum, maximum])
	water = mol.copy()
	water.filter("name OH2")
	num_wat += (water.coords.shape[-1] * water.coords.shape[0])
	for i in range(water.coords.shape[-1]):
		for o in water.coords[:,:,i]:
			a = int(floor(o[0]))
			b = int(floor(o[1]))
			c = int(floor(o[2]))
			try:
				mat[a, b, c] += 1
			except:
				sys.stderr.write("Water outside the box found at coordinates " + str(a) + ", " + str(b) + ", " + str(c) + ". \n")
				num_wat -= 1


	tup = (num_wat, mat)
	return tup

Once this function is run, we define the two outputs of it, because these will be also arguments of the function in further runs. Then, the matrix and the number of waters are being modified every time this function is called.

In [21]:
for element in mol_list:
		res = get_mat(element, mat, num_wat, maximum)
		num_wat = res[0]
		mat = res[1]
		print(num_wat)

Water outside the box found at coordinates -261, 8, 296. 


14131999


Here is the main difference between the python code and this notebook: the assignation of the sigma value for the gaussian correction of the water molecule distribution. For doing this we define a variable named sigma, containing the value in amstrongs of the standard deviation. Then, we introduce our matrix and this sigma value in the gaussian_filter function from scipy. 

In [22]:
from scipy.ndimage.filters import *
sigma = 0.1
mat = gaussian_filter(mat, sigma)
amax(mat)

146.0

Once we have the multidimension array ready, we perform several mathematical transformations on this matrix. First, we replace zero values by 10^-100. Doing this we are avoiding discordant mathematical operations in the following operations of the program. Second, we apply the formula corresponding to the chemical potential of water. 

In [23]:
mat[mat == 0] = 10**-100
mat = -(kb * t * ma.log(mat/num_wat))

Now, we define arrays which determine the extreme values of the grid. We have defined the dimensions of our grid as a function of the maximum x value on the first frame, this value was saved on the variable cord. So we define the maximum point of the array as the coord value for all dimensions, and zero as the minimum point in all directions. We also define an array with the resolution of the grid, which is one amstrong.

In [24]:
max_vec = array([cord, cord, cord])
min_vec = array([0,0,0])
res_vec = array([1,1,1])

Finally, we call the writeVoxel function, which will create a volumetric file containing an isosurface for water distribution in the three dimensional space. In the python scritp there's also a difference here, due the fact that the name of the output .cube file is specified by an argument.

In [25]:
writeVoxels(mat, "isosurface.cube", min_vec, max_vec, res_vec)

Now, the obtained isosurface can be visualized with vmd.